In [98]:
import pandas as pd
import numpy as np
import os
import sys
from scipy.interpolate import CubicSpline
import warnings

pd.options.mode.chained_assignment = None  # default='warn'
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [637]:
# Read the CSV file
df_wind = pd.read_csv(os.path.join(os.getcwd(), '..', 'data', 'wind2.csv'))

# drop the columns that are not needed
df_wind = df_wind.drop(columns=['SS_Price', 'boa_MWh', 'DA_Price', 'Wind_MWh_credit', 'dtm', 'MIP'])

# to datetime
df_wind['reference_time'] = pd.to_datetime(df_wind['reference_time'])
df_wind['valid_time'] = pd.to_datetime(df_wind['valid_time'])
df_latest = df_wind.groupby('valid_time').tail(1)

# drop everything after Jan 20, 2024
df_latest = df_latest[df_latest['valid_time'] < '2024-01-21']

# calculate the average of the two forecasts and drop the individual forecasts (except WindSpeed^3:100)
df_latest['WindSpeed^3:100_avg'] = (df_latest['WindSpeed^3:100_dwd'] + df_latest['WindSpeed^3:100_ncep']) / 2
df_latest['Temperature_avg'] = (df_latest['Temperature_dwd'] + df_latest['Temperature_ncep']) / 2
df_latest['RelativeHumidity_avg'] = (df_latest['RelativeHumidity_dwd'] + df_latest['RelativeHumidity_ncep']) / 2
df_latest['WindDirection:100_avg'] = (df_latest['WindDirection:100_dwd'] + df_latest['WindDirection:100_ncep']) / 2

df_latest = df_latest.drop(columns=['WindSpeed_dwd', 'WindSpeed_ncep', 'Temperature_dwd', 'Temperature_ncep', 'RelativeHumidity_dwd', 'RelativeHumidity_ncep', 'WindSpeed^3_dwd', 'WindSpeed^3_ncep', 'WindDirection_dwd', 'WindDirection_ncep', 'WindSpeed:100_dwd', 'WindSpeed:100_ncep', 'WindSpeed:100^3_dwd', 'WindSpeed:100^3_ncep', 'WindDirection:100_dwd', 'WindDirection:100_ncep'])

# remove ^3 from the column names
df_latest.columns = [col.replace('^3', '') for col in df_latest.columns]

# i. Calculate Power Forecast based on Physics

## 1. Calculate Air Density

In [648]:
# Constants
R_d = 287.05  # Specific gas constant for dry air (J/(kg·K))
R_v = 461.5   # Specific gas constant for water vapor (J/(kg·K))
p = 101325    # Standard atmospheric pressure in Pa

# Assuming df_latest is your original DataFrame and contains 'Temperature_dwd', 'RelativeHumidity_dwd', 'WindSpeed_dwd'
# Convert temperature from Celsius to Kelvin
df_latest['Temperature_K'] = df_latest['Temperature_avg'] + 273.15

# Calculate saturation vapor pressure (using temperature in Celsius), Tetens formula
e_s = 0.61078 * np.exp((17.27 * df_latest['Temperature_avg']) / (df_latest['Temperature_avg'] + 237.3))

# in pa
e_s = 1000 * e_s

# Calculate actual vapor pressure
e = df_latest['RelativeHumidity_avg'] / 100 * e_s

# Calculate air density (ρ) in kg/m³
df_latest['AirDensity'] = (p - e) / (R_d * df_latest['Temperature_K']) + (e / (R_v * df_latest['Temperature_K']))

## 2. Calculate Power based on Density and Wind Speed

In [667]:
def calculate_wind_generation(dataset, rotor_diameter = 154, efficiency = 0.31, limiter = 1, minimum_wind_speed = 3, maximum_wind_speed_for_operation = 25, add_opt = False):
    rotor_area = np.pi * (rotor_diameter / 2) ** 2  # in m²
    
    # turbine requires 3m/s to start rotating
    const_internal_friction_coefficient = 0.5 * 1.240 * rotor_area * 3**3 * efficiency * 174 / 1000000
    maximum_power_per_turbine = 7 # in MW

    # Calculate power output based on wind speed at 100m
    dataset['WindPower:100'] = 0.5 * dataset['AirDensity'] * rotor_area * dataset['WindSpeed:100_avg'] ** 3 * 174 / 1000000
    dataset[f'UsableWindPower{"_opt" if add_opt else ":100"}'] = np.minimum(dataset['WindPower:100'], maximum_power_per_turbine * 174 * limiter / efficiency)
    dataset[f'PowerOutput{"_opt" if add_opt else ""}'] = np.where((dataset['WindSpeed:100_avg'] >= minimum_wind_speed) & (dataset['WindSpeed:100_avg'] <= maximum_wind_speed_for_operation), dataset['UsableWindPower:100'] * efficiency - const_internal_friction_coefficient, 0)

    return dataset

In [668]:
df_latest = calculate_wind_generation(df_latest)

# Absolute error
print('Mean Absolute error:', np.mean(np.abs(df_latest['Wind_MW'] - df_latest['PowerOutput'])))
# Initialize a list to collect the stats
stats_df = pd.DataFrame({'quantile': [], 'quantile_range': [], 'mean_absolute_error': [], 'mean_error': []})

# Mean absolute error in 10% quantiles
for q in np.arange(0.1, 1.1, 0.1):
    # Filter the DataFrame for each quantile range
    quantile_df = df_latest[(df_latest['Wind_MW'] < df_latest['Wind_MW'].quantile(q)) & (df_latest['Wind_MW'] >= df_latest['Wind_MW'].quantile(q - 0.1))]
    
    # Calculate the statistics
    quantile_stats = {
        'quantile': q,
        'quantile_range': f'{df_latest["Wind_MW"].quantile(q - 0.1):.1f} - {df_latest["Wind_MW"].quantile(q):.1f}',
        'mean_absolute_error': np.mean(np.abs(quantile_df['Wind_MW'] - quantile_df['PowerOutput'])),
        'mean_error': np.mean(quantile_df['Wind_MW'] - quantile_df['PowerOutput']),
    }
    
    # Append to the list
    stats_df.loc[len(stats_df)] = quantile_stats

# Display the DataFrame
stats_df

Mean Absolute error: 101.04497857586688


,quantile,quantile_range,mean_absolute_error,mean_error
0,0.1,0.0 - 18.8,39.517385,-37.074092
1,0.2,18.8 - 91.8,39.397084,-9.304232
2,0.3,91.8 - 198.0,70.327971,1.282190
3,0.4,198.0 - 339.9,96.171174,20.831775
4,0.5,339.9 - 518.0,135.894088,40.554842
5,0.6,518.0 - 748.8,174.372297,69.961393
6,0.7,748.8 - 994.1,201.695335,59.253460
7,0.8,994.1 - 1108.8,141.223749,-26.494254
8,0.9,1108.8 - 1154.5,79.295350,-39.540040
9,1.0,1154.5 - 1192.7,32.586704,-21.556105


## 3. Find optimal $\eta$ to update the approximation of the turbine efficiency

In [669]:
errors = {}
for efficiency in np.arange(0.28, 0.35, 0.001):
    for limiter in np.arange(0.9, 1.01, 0.01):
        df_tmp = calculate_wind_generation(df_latest, efficiency=efficiency, limiter=limiter)
        mean_abs_error = np.mean(np.abs(df_tmp['Wind_MW'] - df_tmp['PowerOutput']))
        mean_error = np.mean(df_tmp['Wind_MW'] - df_tmp['PowerOutput'])
        errors[(efficiency.round(3), limiter.round(2))] = [mean_abs_error, mean_error]

In [670]:
# Find the minimum error
min_error = min(errors, key=lambda x: errors[x])
print(f'Minimum mean absolute error: {errors[min_error][0]:.2f} with efficiency={min_error}')
print(f'Mean error: {errors[min_error][1]:.2f}')

Minimum mean absolute error: 88.20 with efficiency=(0.348, 0.94)
Mean error: -1.87


In [671]:
efficiency = min_error[0]
limiter = min_error[1]

df_latest = calculate_wind_generation(df_latest, efficiency=efficiency, limiter=limiter, add_opt = True)

In [672]:
# Update stats_df using optimal values
for q in np.arange(0.1, 1.1, 0.1):
    quantile_df = df_latest[(df_latest['Wind_MW'] < df_latest['Wind_MW'].quantile(q)) & (df_latest['Wind_MW'] >= df_latest['Wind_MW'].quantile(q - 0.1))]
    quantile_row = {
        'quantile': q,
        'mean_absolute_error_opt': np.mean(np.abs(quantile_df['Wind_MW'] - quantile_df['PowerOutput_opt'])),
        'mean_error_opt': np.mean(quantile_df['Wind_MW'] - quantile_df['PowerOutput_opt'])
    }
    stats_df.loc[stats_df['quantile'] == q, 'mean_absolute_error_opt'] = quantile_row['mean_absolute_error_opt']
    stats_df.loc[stats_df['quantile'] == q, 'mean_error_opt'] = quantile_row['mean_error_opt']

stats_df

,quantile,quantile_range,mean_absolute_error,mean_error,mean_absolute_error_opt,mean_error_opt
0,0.1,0.0 - 18.8,39.517385,-37.074092,42.121918,-39.758370
1,0.2,18.8 - 91.8,39.397084,-9.304232,43.399286,-16.882116
2,0.3,91.8 - 198.0,70.327971,1.282190,73.406415,-14.891074
3,0.4,198.0 - 339.9,96.171174,20.831775,98.300081,-8.363622
4,0.5,339.9 - 518.0,135.894088,40.554842,134.949444,-3.775859
5,0.6,518.0 - 748.8,174.372297,69.961393,167.986107,6.596135
6,0.7,748.8 - 994.1,201.695335,59.253460,191.051311,-15.665951
7,0.8,994.1 - 1108.8,141.223749,-26.494254,131.521919,-64.785121
8,0.9,1108.8 - 1154.5,79.295350,-39.540040,68.569076,-47.452606
9,1.0,1154.5 - 1192.7,32.586704,-21.556105,25.346413,-19.268428


## 3. Make Forecast more accurate using smoothing of windspeed between datapoints

In [673]:
# Iterate over each reference_time
df_latest['PowerOutput_smoothed'] = 0.0
rotor_diameter = 154
minimum_wind_speed = 3
maximum_wind_speed_for_operation = 25
rotor_area = np.pi * (rotor_diameter / 2) ** 2
const_internal_friction_coefficient = 0.5 * 1.240 * rotor_area * 3**3 * efficiency * 174 / 1000000
maximum_power_per_turbine = 7 # in MW

for reference_time, group in df_latest.groupby('reference_time'):

    # Get the valid_times and corresponding wind speeds for this reference_time
    valid_times = group['valid_time'].values
    wind_speeds = group['WindSpeed:100_avg'].values
    actual_generation = group['Wind_MW'].values
    
    # Convert valid_times to seconds for interpolation
    valid_times_in_seconds = (valid_times - valid_times[0]).astype('timedelta64[m]').astype(int) * 60  # seconds
    
    if len(valid_times) < 2:
        wind_power = 0.5 * group['AirDensity'].iloc[0] * rotor_area * avg_cubed_wind * 174 / 1000000
        usable_wind_power = min(wind_power, 7 * 174 * limiter / efficiency)
        power_output = usable_wind_power * efficiency - const_internal_friction_coefficient if (group['WindSpeed:100_avg'].iloc[0] >= minimum_wind_speed) and (group['WindSpeed:100_avg'].iloc[0] <= maximum_wind_speed_for_operation) else 0

        # Update PowerOutput
        df_latest.loc[(df_latest['reference_time'] == reference_time) & (df_latest['valid_time'] == valid_times[0]), 'PowerOutput_smoothed'] = power_output
        continue
    # Perform cubic spline interpolation for 1-minute intervals
    wind_speed_spline = CubicSpline(valid_times_in_seconds, wind_speeds)
    
    # Generate 1-minute intervals for the valid_time range
    for i in range(len(valid_times) - 1):
        t_start = valid_times_in_seconds[i]
        t_end = valid_times_in_seconds[i + 1]
        
        # Generate time points at 1-minute intervals within this 30-minute window
        times_1min = np.arange(t_start, t_end, 60)
        
        # Interpolate wind speeds at 1-minute intervals
        interpolated_wind_speeds = wind_speed_spline(times_1min)
        
        # Calculate the average wind power using the cubed wind speeds, reduce speeds <3ms to 0
        avg_cubed_wind = np.mean(np.where((interpolated_wind_speeds >= 3) & (interpolated_wind_speeds <= 25), interpolated_wind_speeds, 0) ** 3)
        avg_cubed_wind = np.mean(interpolated_wind_speeds ** 3)
        # get frac with windspeed > 3 and < 25
        frac_generation = np.mean(np.where((interpolated_wind_speeds >= 3) & (interpolated_wind_speeds <= 25), 1, 0))
        
        # Calculate wind power and apply limits for each interval
        wind_power = 0.5 * group['AirDensity'].iloc[i] * rotor_area * avg_cubed_wind * 174 / 1000000
        usable_wind_power = min(wind_power, 7 * 174 * limiter / efficiency)
        
        # Calculate final power output based on cut-in, cut-out wind speeds and efficiency
        power_output = max(0, usable_wind_power * efficiency - const_internal_friction_coefficient * frac_generation)
        
        # Update PowerOutput
        valid_time = pd.to_datetime(valid_times[i]).tz_localize('UTC')
        df_latest.loc[(df_latest['reference_time'] == reference_time) & (df_latest['valid_time'] == valid_time), 'PowerOutput_smoothed'] = power_output

In [674]:
# Absolute error
print('Mean Absolute error:', np.mean(np.abs(df_latest['Wind_MW'] - df_latest['PowerOutput_smoothed'])))
# Mean absolute error in 10% quantiles
for q in np.arange(0.1, 1.1, 0.1):
    quantile_df = df_latest[(df_latest['Wind_MW'] < df_latest['Wind_MW'].quantile(q)) & (df_latest['Wind_MW'] >= df_latest['Wind_MW'].quantile(q - 0.1))]
    quantile_row = {
        'quantile': q,
        'mean_absolute_error_new': np.mean(np.abs(quantile_df['Wind_MW'] - quantile_df['PowerOutput_smoothed'])),
        'mean_error_new': np.mean(quantile_df['Wind_MW'] - quantile_df['PowerOutput_smoothed'])
    }
    stats_df.loc[stats_df['quantile'] == q, 'mean_absolute_error_new'] = quantile_row['mean_absolute_error_new']
    stats_df.loc[stats_df['quantile'] == q, 'mean_error_new'] = quantile_row['mean_error_new']

stats_df


Mean Absolute error: 127.42721981203687


,quantile,quantile_range,mean_absolute_error,mean_error,mean_absolute_error_opt,mean_error_opt,mean_absolute_error_new,mean_error_new
0,0.1,0.0 - 18.8,39.517385,-37.074092,42.121918,-39.758370,41.318240,-39.899149
1,0.2,18.8 - 91.8,39.397084,-9.304232,43.399286,-16.882116,43.325882,-11.178727
2,0.3,91.8 - 198.0,70.327971,1.282190,73.406415,-14.891074,77.817429,-1.788819
3,0.4,198.0 - 339.9,96.171174,20.831775,98.300081,-8.363622,110.617502,15.959677
4,0.5,339.9 - 518.0,135.894088,40.554842,134.949444,-3.775859,156.091264,33.596578
5,0.6,518.0 - 748.8,174.372297,69.961393,167.986107,6.596135,200.108349,58.120736
6,0.7,748.8 - 994.1,201.695335,59.253460,191.051311,-15.665951,229.584225,64.320030
7,0.8,994.1 - 1108.8,141.223749,-26.494254,131.521919,-64.785121,163.040286,75.216770
8,0.9,1108.8 - 1154.5,79.295350,-39.540040,68.569076,-47.452606,113.719137,109.133248
9,1.0,1154.5 - 1192.7,32.586704,-21.556105,25.346413,-19.268428,138.687305,138.687305


## => It got worse????

# ii. Modeling the residium 

## 1. XGBoost

In [675]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate the residuals (difference between actual and forecast)
df_latest['residual'] = df_latest['Wind_MW'] - df_latest['PowerOutput_opt']

# add three lag features for WindSpeed:100_dwd
df_latest['WindSpeed:100_dwd_lag1'] = df_latest['WindSpeed:100_dwd'].shift(1)
df_latest['WindSpeed:100_dwd_lag2'] = df_latest['WindSpeed:100_dwd'].shift(2)
df_latest['WindSpeed:100_dwd_lag3'] = df_latest['WindSpeed:100_dwd'].shift(3)

df_latest = df_latest.dropna()

# Define the features (X) and the target (y)
X = df_latest[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity', 'UsableWindPower_opt', 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3']]  # Add any other relevant features
y = df_latest['residual']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [73]:
# Initialize the XGBoost regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', seed=42)

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.01, 0.1],  # Controls the step size in each boosting step
    'n_estimators': [100, 500, 1000],  # Number of boosting rounds
    'max_depth': [5, 7],  # Maximum depth of the trees
    'subsample': [0.7, 1.0],  # Proportion of training data used for fitting the individual trees
    'colsample_bytree': [0.7, 1.0],  # Proportion of features used for fitting individual trees
    'gamma': [0.1, 0.3],  # Minimum loss reduction required to make a further partition on a leaf node
    'reg_lambda': [1, 10],  # L2 regularization term
    'reg_alpha': [0, 1],  # L1 regularization term
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_xgb_model = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)


Fitting 3 folds for each of 384 candidates, totalling 1152 fits
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.7; total time=   0.3s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.7; total time=   0.3s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=1.0; total time=   0.3s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=1.0; total time=   0.3s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.7; total time=   0.3s
[CV] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=1.0; total time=   0.3s
[CV] END col

In [74]:
# Predict the residuals for the test set
y_pred = best_xgb_model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'Mean Absolute Error (MAE): {mae:.3f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.3f}')


Mean Absolute Error (MAE): 75.430
Root Mean Squared Error (RMSE): 127.758


In [75]:
# compared to always predicting 0
y_pred_baseline = np.zeros_like(y_test)
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_pred_baseline))

print(f'Mean Absolute Error (Baseline): {mae_baseline:.3f}')
print(f'Root Mean Squared Error (Baseline): {rmse_baseline:.3f}')

Mean Absolute Error (Baseline): 95.688
Root Mean Squared Error (Baseline): 160.782


# Training quantile Models

In [676]:
# Calculate the residuals (difference between actual and forecast)
df_latest['residual'] = df_latest['Wind_MW'] - df_latest['PowerOutput_opt']

# add three lag features for WindSpeed:100_dwd
df_latest['WindSpeed:100_dwd_lag1'] = df_latest['WindSpeed:100_dwd'].shift(1)
df_latest['WindSpeed:100_dwd_lag2'] = df_latest['WindSpeed:100_dwd'].shift(2)
df_latest['WindSpeed:100_dwd_lag3'] = df_latest['WindSpeed:100_dwd'].shift(3)

df_latest = df_latest.dropna()

## GradientBoostingRegressor

In [754]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor

def train_gbr_model_quantiles(dataset, residual = True, include_calculation_features = False, eval = False, shuffle = False, split = 0.2):
    # Define the features (X) and the target (y)
    X = dataset[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity', 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3']]
    if include_calculation_features and not residual:
        X['PowerOutput_opt'] = dataset['PowerOutput_opt']
        X['UsableWindPower_opt'] = dataset['UsableWindPower_opt']
    if residual:
        X['UsableWindPower_opt'] = dataset['UsableWindPower_opt']
    y = dataset['residual'] if residual else dataset['Wind_MW']

    # Split the data into training and test sets
    if split > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42, shuffle=shuffle)
    else:
        X_train, y_train = X, y

    # Define the quantiles we want to model
    quantiles = np.arange(0.1, 1.0, 0.1)

    # Dictionary to hold the best models for each quantile
    best_models = {}

    # Loop over each quantile and train a model
    for quantile in quantiles:
        print(f"Training model for {quantile * 100:.0f}% quantile...")
        
        # Initialize the Gradient Boosting Regressor with the quantile loss
        gbr = HistGradientBoostingRegressor(loss='quantile', quantile=quantile, random_state=42)
        
        # Define the parameter grid for hyperparameter tuning
        param_grid = {
            'learning_rate': [0.01, 0.1],  # Controls the step size in each boosting step
            'max_iter': [300, 500],  # Number of boosting rounds
            'max_depth': [3, 5, 7],  # Maximum depth of the trees
            'min_samples_leaf': [1, 5, 10],  # Minimum number of samples required to be at a leaf node
        }

        # Custom scoring function for quantile regression (Pinball loss)
        def pinball_loss(y_true, y_pred):
            delta = y_true - y_pred
            return np.mean(np.maximum(quantile * delta, (quantile - 1) * delta))
        
        pinball_scorer = make_scorer(pinball_loss, greater_is_better=False)
        
        # Initialize GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1, scoring=pinball_scorer)
        
        # Fit the model to the training data
        grid_search.fit(X_train, y_train)
        
        # Store the best model for this quantiles
        best_models[quantile] = grid_search.best_estimator_
        
        print(f"Best parameters for {quantile * 100:.0f}% quantile: {grid_search.best_params_}")

    if eval and split > 0:
        # Loop over each quantile to evaluate the models
        for quantile, model in best_models.items():
            # Predict the residuals for the test set
            y_pred = model.predict(X_test)
            
            # Calculate the evaluation metrics
            me = np.mean(y_test - y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            pinball = pinball_loss(y_test, y_pred)
            
            print(f"Evaluation for {quantile * 100:.0f}% quantile:")
            print(f"Mean Error (ME): {me:.3f}")
            print(f"Mean Absolute Error (MAE): {mae:.3f}")
            print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
            print(f"Pinball Loss: {pinball:.3f}")

            # compare to training data
            y_pred_train = model.predict(X_train)
            me = np.mean(y_train - y_pred_train)
            mae = mean_absolute_error(y_train, y_pred_train)
            rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
            pinball = pinball_loss(y_train, y_pred_train)

            print(f"\nTraining data:")
            print(f"Mean Error (ME): {me:.3f}")
            print(f"Mean Absolute Error (MAE): {mae:.3f}")
            print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
            print(f"Pinball Loss: {pinball:.3f}")
            print("-" * 40)

    for quantile, model in best_models.items():
        joblib.dump(model, os.path.join(os.getcwd(), 'models', f'gbr_quantile_{quantile:.1f}_res-{residual}_calc-{include_calculation_features}.pkl'))
    
    return best_models



In [ ]:
best_models_gbr = train_gbr_model_quantiles(df_latest, eval=True, )

# XGBoost

In [757]:
def train_xgb_model_quantiles(dataset, residual = True, include_calculation_features = False, eval = False, shuffle = False, split = 0.2):
    # Define the features (X) and the target (y)
    X = dataset[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity', 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3']]
    if include_calculation_features and not residual:
        X['PowerOutput_opt'] = dataset['PowerOutput_opt']
        X['UsableWindPower_opt'] = dataset['UsableWindPower_opt']
    if residual:
        X['UsableWindPower_opt'] = dataset['UsableWindPower_opt']

    y = dataset['residual'] if residual else dataset['Wind_MW']

    # Split the data into training and test sets
    if split > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42, shuffle=shuffle)
    else:
        X_train, y_train = X, y

    # Define the quantiles we want to model
    quantiles = np.arange(0.1, 1.0, 0.1)

    # Dictionary to hold the best models for each quantile
    best_models = {}

    # Loop over each quantile and train a model
    for quantile in quantiles:
        print(f"Training model for {quantile * 100:.0f}% quantile...")
        
        # Initialize the XGBoost Regressor with the quantile loss and GPU support
        xg_reg = xgb.XGBRegressor(objective='reg:quantile', alpha=quantile, tree_method='gpu_hist', random_state=42, verbosity=1)
        
        # Define the parameter grid for hyperparameter tuning
        param_grid = {
            'learning_rate': [0.01, 0.1],
            'n_estimators': [300, 500],
            'max_depth': [3, 5, 7],
            'subsample': [0.7],
            'colsample_bytree': [0.7, 1.0]
        }

        # Custom scoring function for quantile regression (Pinball loss)
        def pinball_loss(y_true, y_pred):
            delta = y_true - y_pred
            return np.mean(np.maximum(quantile * delta, (quantile - 1) * delta))
        
        pinball_scorer = make_scorer(pinball_loss, greater_is_better=False)
        
        # Initialize GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring=pinball_scorer)
        
        # Fit the model to the training data
        grid_search.fit(X_train, y_train)
        
        # Store the best model for this quantile
        best_models[quantile] = grid_search.best_estimator_
        
        print(f"Best parameters for {quantile * 100:.0f}% quantile: {grid_search.best_params_}")
    
    if eval:
        # Loop over each quantile to evaluate the models
        for quantile, model in best_models.items():
            # Predict the residuals for the test set
            y_pred = model.predict(X_test)
            
            # Calculate the evaluation metrics
            me = np.mean(y_test - y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            pinball = pinball_loss(y_test, y_pred)
            
            print(f"Evaluation for {quantile * 100:.0f}% quantile:")
            print(f"Mean Error (ME): {me:.3f}")
            print(f"Mean Absolute Error (MAE): {mae:.3f}")
            print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
            print(f"Pinball Loss: {pinball:.3f}")
            print("-" * 40)

            # compare to training data
            y_pred_train = model.predict(X_train)
            me = np.mean(y_train - y_pred_train)
            mae = mean_absolute_error(y_train, y_pred_train)
            rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
            pinball = pinball_loss(y_train, y_pred_train)

            print(f"\nTraining data:")
            print(f"Mean Error (ME): {me:.3f}")
            print(f"Mean Absolute Error (MAE): {mae:.3f}")
            print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
            print(f"Pinball Loss: {pinball:.3f}")
            print("-" * 40)
    
    for quantile, model in best_models.items():
        joblib.dump(model, os.path.join(os.getcwd(), 'models', f'xgboost_quantile_{quantile:.1f}_res-{residual}_calc-{include_calculation_features}.pkl'))
    
    return best_models

In [ ]:
best_models_xgb = train_xgb_model_quantiles(df_latest)

# Exploring the performance for sample dates

In [740]:
import joblib
import os
import plotly.graph_objs as go
import numpy as np
from IPython.display import display

def create_forecast_for_dates(start_day, end_day = None, all = False, gbr = False, xgb = False, show_data = False, res = True, calc = False):
    if all == True:
        gbr = True
        xgb = True
    if end_day is None:
        end_day = start_day
    
    # Define the quantiles
    quantiles = np.arange(0.1, 1.0, 0.1)

    # Now, make predictions using the loaded models for your sample data
    start_date = pd.to_datetime(np.datetime64(start_day)).tz_localize('UTC')
    end_date = pd.to_datetime(np.datetime64(end_day)).tz_localize('UTC')
    sample_data = df_latest[((df_latest['valid_time'].dt.date >= start_date.date()) & (df_latest['valid_time'].dt.date <= end_date.date()))]

    # Prepare the feature data for prediction from the sample_data
    X_sample = sample_data[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity', 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3']]
    if calc and not res:
        X_sample['PowerOutput_opt'] = sample_data['PowerOutput_opt']
        X_sample['UsableWindPower_opt'] = sample_data['UsableWindPower_opt']
    
    if res:
        # Also use the calculated UseableWindPower when predicting the residuals
        X_sample['UsableWindPower_opt'] = sample_data['UsableWindPower_opt']

    # Real data (Wind_MW)
    y_real = sample_data['Wind_MW'].values

    # Add custom data for hover
    custom_data = sample_data[['WindSpeed:100_dwd', 'PowerOutput_opt', 'WindSpeed:100_ncep']]

    # Plot GBR quantile predictions
    if gbr:
        best_models_gbr = {}
        for quantile in quantiles:
            model_path_gbr = os.path.join(os.getcwd(), 'models', f'gbr_quantile_{quantile:.1f}_res-{res}_calc-{calc}.pkl')
            best_models_gbr[quantile] = joblib.load(model_path_gbr)
        gbr_predictions = {}
        for quantile in quantiles:
            gbr_predictions[quantile] = best_models_gbr[quantile].predict(X_sample) + sample_data['PowerOutput_opt'] if res else best_models_gbr[quantile].predict(X_sample)

        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(
            x=sample_data['valid_time'], 
            y=y_real, 
            mode='lines+markers', 
            name='Real Data (Wind_MW)', 
            line=dict(color='black'),
            customdata=custom_data,  # Add custom data to hover
            hovertemplate='Wind MW: %{y:.3f}<br>PowerOutput_opt: %{customdata[1]:.3f}<br>WindSpeed:100_dwd: %{customdata[0]:.3f}<br>WindSpeed:100_ncpe: %{customdata[2]:.3f}<extra></extra>'
        ))
        for quantile in quantiles:
            fig1.add_trace(go.Scatter(
                x=sample_data['valid_time'],
                y=gbr_predictions[quantile],
                mode='lines',
                name=f'GBR {int(quantile * 100)}th Percentile',
                line=dict(dash='dash')
            ))

        fig1.update_layout(
            title=f"Quantile Predictions (GradientBoosting) on {start_date.date()} to {end_date.date()}",
            xaxis_title="Valid Time",
            yaxis_title="Power Output (MW)",
            hovermode="x unified",
            legend_title="Models"
        )

        fig1.show()
    
    # Plot XGB quantile predictions
    if xgb:
        best_models_xgb = {}
        for quantile in quantiles:
            model_path_xgb = os.path.join(os.getcwd(), 'models', f'xgboost_quantile_{quantile:.1f}_res-{res}_calc-{calc}.pkl')
            best_models_xgb[quantile] = joblib.load(model_path_xgb)

        xgb_predictions = {}
        for quantile in quantiles:
            xgb_predictions[quantile] = best_models_xgb[quantile].predict(X_sample) + sample_data['PowerOutput']

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(
            x=sample_data['valid_time'], 
            y=y_real, 
            mode='lines+markers', 
            name='Real Data (Wind_MW)', 
            line=dict(color='black'),
            customdata=custom_data,  # Add custom data to hover
            hovertemplate='Wind MW: %{y:.3f}<br>PowerOutput_opt: %{customdata[1]:.3f}<br>WindSpeed:100_dwd: %{customdata[0]:.3f}<extra></extra>'
        ))
        for quantile in quantiles:
            fig2.add_trace(go.Scatter(
                x=sample_data['valid_time'],
                y=xgb_predictions[quantile],
                mode='lines',
                name=f'XGB {int(quantile * 100)}th Percentile',
                line=dict(dash='dash')
            ))

        fig2.update_layout(
            title=f"Quantile Predictions (XGBoost) from {start_date.date()} to {end_date.date()}",
            xaxis_title="Valid Time",
            yaxis_title="Power Output (MW)",
            hovermode="x unified",
            legend_title="Models"
        )

        fig2.show()
    if show_data:
        X_sample['PowerOutput_opt'] = sample_data['PowerOutput_opt']
        X_sample['timestamp'] = sample_data['valid_time']
        pd.set_option('display.max_rows', None)
        display(X_sample)
        pd.set_option('display.max_rows', 10)


In [205]:
create_forecast_for_dates('2023-11-01', gbr=True)

In [704]:
start_day = df_latest['valid_time'].dt.date.min()

In [706]:
create_forecast_for_dates(start_day, start_day + pd.DateOffset(days=3), gbr=True)
start_day = start_day + pd.DateOffset(days=4)

In [734]:
start_day = df_latest['valid_time'].dt.date.min()

In [736]:
create_forecast_for_dates(start_day, start_day + pd.DateOffset(days=3), gbr=True)
start_day = start_day + pd.DateOffset(days=4)

## When looking through the ouputs, it seems wise to delete the upper limit for power generation from entirely to get rid o the extreme forecasts

In [730]:
df_latest = calculate_wind_generation(df_latest, efficiency=efficiency, limiter=limiter, add_opt = True, maximum_wind_speed_for_operation = np.inf)
df_latest['residual'] = df_latest['Wind_MW'] - df_latest['PowerOutput_opt']

# add three lag features for WindSpeed:100_dwd
df_latest['WindSpeed:100_dwd_lag1'] = df_latest['WindSpeed:100_dwd'].shift(1)
df_latest['WindSpeed:100_dwd_lag2'] = df_latest['WindSpeed:100_dwd'].shift(2)
df_latest['WindSpeed:100_dwd_lag3'] = df_latest['WindSpeed:100_dwd'].shift(3)

df_latest = df_latest.dropna()

In [731]:
df_training_set = df_latest[['valid_time', 'reference_time', 'WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity', 'UsableWindPower_opt', 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3', 'residual', 'PowerOutput_opt', 'Wind_MW']]

In [732]:
df_problematic_datapoints = pd.read_csv(os.path.join(os.getcwd(), '..', 'data', 'wind_problematic_datapoints.csv'))

# Convert 'start_date' and 'end_date' to datetime format
df_problematic_datapoints['start_time'] = pd.to_datetime(df_problematic_datapoints['start_time']).dt.tz_localize('UTC')
df_problematic_datapoints['end_time'] = pd.to_datetime(df_problematic_datapoints['end_time']).dt.tz_localize('UTC')

print(df_training_set.shape)

# Ensure the 'valid_time' column in df_training_set is in datetime format
df_training_set['valid_time'] = pd.to_datetime(df_training_set['valid_time'])

# Loop over each date range in df_problematic_datapoints and filter out the rows in df_training_set
for idx, row in df_problematic_datapoints.iterrows():
    start_date = row['start_time']
    end_date = row['end_time']
    
    # Filter out rows that fall within the current date range
    df_training_set = df_training_set[~((df_training_set['valid_time'] >= start_date) & (df_training_set['valid_time'] <= end_date))]

print(df_training_set.shape)
df_training_set.to_csv(os.path.join(os.getcwd(), '..', 'data', 'wind_training_set.csv'), index=False)

(57850, 13)
(57073, 13)


# Re-training of the models

### GBR Models

In [750]:
best_models_gbr = train_gbr_model_quantiles(df_training_set, residual = True, include_calculation_features = False, eval=True)

Training model for 10% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 10% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 1}
Training model for 20% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 20% quantile: {'learning_rate': 0.01, 'max_depth': 5, 'max_iter': 300, 'min_samples_leaf': 10}
Training model for 30% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 30% quantile: {'learning_rate': 0.01, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 10}
Training model for 40% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 40% quantile: {'learning_rate': 0.01, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 5}
Training model for 50% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 50% quantile: {'learning_rate': 0.01, 

In [751]:
best_models_gbr = train_gbr_model_quantiles(df_training_set, residual = False, include_calculation_features = False, eval=True)

Training model for 10% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 10% quantile: {'learning_rate': 0.01, 'max_depth': 7, 'max_iter': 500, 'min_samples_leaf': 5}
Training model for 20% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 20% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 10}
Training model for 30% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 30% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 300, 'min_samples_leaf': 10}
Training model for 40% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 40% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 300, 'min_samples_leaf': 1}
Training model for 50% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 50% quantile: {'learning_rate': 0.01, 'm

In [753]:
best_models_gbr = train_gbr_model_quantiles(df_training_set, residual = False, include_calculation_features = True, eval=True)

Training model for 10% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 10% quantile: {'learning_rate': 0.01, 'max_depth': 5, 'max_iter': 500, 'min_samples_leaf': 10}
Training model for 20% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 20% quantile: {'learning_rate': 0.01, 'max_depth': 5, 'max_iter': 500, 'min_samples_leaf': 10}
Training model for 30% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 30% quantile: {'learning_rate': 0.01, 'max_depth': 5, 'max_iter': 500, 'min_samples_leaf': 1}
Training model for 40% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 40% quantile: {'learning_rate': 0.01, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 10}
Training model for 50% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 50% quantile: {'learning_rate': 0.01

# The best model is the RTCF model. Lets retrain it on the full dataset to obtain the model to use for the actual forecast

In [758]:
train_gbr_model_quantiles(df_training_set, residual = True, include_calculation_features = True, eval=False, split = 0)

Training model for 10% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 10% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 300, 'min_samples_leaf': 1}
Training model for 20% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 20% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 10}
Training model for 30% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 30% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 1}
Training model for 40% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 40% quantile: {'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 1}
Training model for 50% quantile...
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for 50% quantile: {'learning_rate': 0.1, 'max_

{0.1: HistGradientBoostingRegressor(loss='quantile', max_depth=3, max_iter=300,
                               min_samples_leaf=1, quantile=0.1,
                               random_state=42),
 0.2: HistGradientBoostingRegressor(loss='quantile', max_depth=3, max_iter=500,
                               min_samples_leaf=10, quantile=0.2,
                               random_state=42),
 0.30000000000000004: HistGradientBoostingRegressor(loss='quantile', max_depth=3, max_iter=500,
                               min_samples_leaf=1, quantile=0.30000000000000004,
                               random_state=42),
 0.4: HistGradientBoostingRegressor(loss='quantile', max_depth=3, max_iter=500,
                               min_samples_leaf=1, quantile=0.4,
                               random_state=42),
 0.5: HistGradientBoostingRegressor(loss='quantile', max_depth=3, max_iter=300,
                               min_samples_leaf=10, quantile=0.5,
                               random_state=4